In [2]:
import pandas as pd
import numpy as np

data=pd.read_csv("../datasets/onlinefraud.csv")
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [3]:
data.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [4]:
data.rename(columns={"oldbalanceOrg":"oldbalanceOrig"},inplace=True)

In [5]:
data=data.drop(["nameOrig","nameDest"],axis=1)

In [6]:
data.head()

,step,type,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1,0
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [7]:
payTypes=data["type"].unique()
print(payTypes)
print(np.where(payTypes=="TRANSFER")[0][0])

['PAYMENT' 'TRANSFER' 'CASH_OUT' 'DEBIT' 'CASH_IN']
1


In [8]:
def changeType(type):
    index=np.where(payTypes==type)[0][0]
    return (index+1)

In [9]:
data["typeInt"]=data["type"].apply(changeType)

In [10]:
data=data.iloc[:,[0,1,9,2,3,4,5,6,7,8]]

In [11]:
data.head()

,step,type,typeInt,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,1,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,1,PAYMENT,1,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,1,TRANSFER,2,181.00,181.0,0.00,0.0,0.0,1,0
3,1,CASH_OUT,3,181.00,181.0,0.00,21182.0,0.0,1,0
4,1,PAYMENT,1,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [12]:
x=np.array(data[["typeInt","amount","oldbalanceOrig","newbalanceOrig","oldbalanceDest","newbalanceDest"]])
y=np.array(data["isFraud"])

print(type(x))
print(type(y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
decisionTree=DecisionTreeClassifier()
decisionTree.fit(xtrain,ytrain)

DecisionTreeClassifier()

In [14]:
from sklearn.metrics import accuracy_score
ypred=decisionTree.predict(xtest)
accuracyDT=accuracy_score(ytest,ypred)*100
print(f"Accuracy : {accuracyDT:.3f}")

Accuracy : 99.973


In [15]:
import pickle

pickle.dump(data,open("../datasets/cleanfrauds.pkl",'wb'))

In [16]:
import pickle

df=pickle.load(open("../datasets/cleanfrauds.pkl","rb"))
df.head()

,step,type,typeInt,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,1,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,1,PAYMENT,1,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,1,TRANSFER,2,181.00,181.0,0.00,0.0,0.0,1,0
3,1,CASH_OUT,3,181.00,181.0,0.00,21182.0,0.0,1,0
4,1,PAYMENT,1,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [17]:
pickle.dump(decisionTree,open("../predictions/decisionTreeModel.pkl","wb"))

In [18]:
dTree=pickle.load(open("../predictions/decisionTreeModel.pkl","rb"))

In [19]:
ypred=dTree.predict(xtest)
accuracy=accuracy_score(ytest,ypred)
print(accuracy)

0.999732814469511


In [20]:
pred=dTree.predict([[1,2,3,4,5,6],[1,2,3,4,5,6],[1,2,3,4,5,6]])
print(pred)

[0 0 0]
